## Remote Training - Azure Machine Learning Service

In [3]:
import azureml.core
from azureml.core import Workspace

# print core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.0.18


### Connect to Workspace

In [4]:
# load workspace configuration from the config.json file in the current folder.
ws = Workspace.from_config()
print(ws.name, ws.location, ws.resource_group, ws.location, sep='\t')

Found the config file in: /Users/Rithin/Desktop/Projects/session-based-vehicle-recommendations/model/config.json
ML-Workspace	westeurope	ML-DEV	westeurope


### Creating an Experiment

In [5]:
experiment_name = 'vehicle-views-all-sessions'

from azureml.core import Experiment
exp = Experiment(workspace=ws, name=experiment_name)

### Create Compute Resource - Run Based

In [ ]:
"""
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.runconfig import RunConfiguration

# list the supported VM families for Azure Machine Learning Compute
#print(AmlCompute.supported_vmsizes(workspace=ws))


# Create a new runconfig object
run_temp_compute = RunConfiguration()

# Signal that you want to use AmlCompute to execute the script
run_temp_compute.target = "amlcompute"

# AmlCompute is created in the same region as your workspace
# Set the VM size for AmlCompute from the list of supported_vmsizes
run_temp_compute.amlcompute.vm_size = 'Standard_NC24'
"""

In [ ]:
"""
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
import os

# choose a name for your cluster
compute_name = os.environ.get("AML_COMPUTE_CLUSTER_NAME", "gpucluster")
compute_min_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MIN_NODES", 0)
compute_max_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MAX_NODES", 4)

# This example uses CPU VM. For using GPU VM, set SKU to STANDARD_NC6
vm_size = os.environ.get("AML_COMPUTE_CLUSTER_SKU", "STANDARD_NC6")


if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print('found compute target. just use it. ' + compute_name)
else:
    print('creating a new compute target...')
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = vm_size,
                                                                min_nodes = compute_min_nodes, 
                                                                max_nodes = compute_max_nodes)

    # create the cluster
    compute_target = ComputeTarget.create(ws, compute_name, provisioning_config)
    
    # can poll for a minimum number of nodes and for a specific timeout. 
    # if no min node count is provided it will use the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
     # For a more detailed view of current AmlCompute status, use get_status()
    print(compute_target.get_status().serialize())
"""

### Get and Upload data for training in Cloud

In [12]:
data_folder = os.path.join(os.getcwd(), 'data')
os.makedirs(data_folder, exist_ok=True)

ds = ws.get_default_datastore()
print(ds.datastore_type, ds.account_name, ds.container_name)

#ds.upload(src_dir=data_folder, target_path='datasets', overwrite=True, show_progress=True)

AzureBlob mlworkspace0237397596 azureml-blobstore-b63b2fa4-c464-45f5-b678-4f311b598992


### Train on Remote Cluster

#### Create Directory and Copy Training Scripts

In [13]:
import os
script_folder = os.path.join(os.getcwd(), "training_scripts")
os.makedirs(script_folder, exist_ok=True)

In [14]:
import shutil
shutil.copy('main.py', script_folder)
shutil.copy('model.py', script_folder)
shutil.copy('utils.py', script_folder)

'/Users/Rithin/Desktop/Projects/session-based-vehicle-recommendations/model/training_scripts/utils.py'

In [ ]:
print(ds.path('datasets/dataset_25000_sessions').as_mount())

#### Create an estimator

In [15]:
from azureml.train.dnn import PyTorch

script_params = {
    '--dataset_folder': ds.path('datasets/dataset_all_sessions').as_mount(),
    '--batchSize':200,
    '--hiddenSize':200,
    '--epoch':15,
    '--lr':0.005,
    '--lr_dc':0.1,
    '--lr_dc_step':3,
    '--top_k':40
}

pt_est = PyTorch(source_directory=script_folder,
                 script_params=script_params,
                 compute_target='amlcompute',
                 vm_size='Standard_NC24',
                 entry_script='main.py',
                 use_gpu=True)

#### Submit the training job to the cluster

In [ ]:
from azureml.core import ScriptRunConfig

run = exp.submit(config=pt_est)

In [17]:
run

Experiment,Id,Type,Status,Details Page,Docs Page
vehicle-views-all-sessions,vehicle-views-all-sessions_1554342148_72dceaa0,azureml.scriptrun,Running,Link to Azure Portal,Link to Documentation


In [ ]:
print(run.get_metrics())

In [ ]:
print(run.get_file_names())

### Register Model

In [27]:
output_folder = "outputs"
model_name = "vehicle_recommendations_model"
item_mapping_name = f'{model_name}_item_veh_mapping'
veh_mapping_name = f'{model_name}_veh_item_mapping'

model_gnn = run.register_model(model_name=model_name, 
                           model_path=f'{output_folder}/{model_name}.pt',
                           tags={"data": "vehicle_clicks", "model": "recommendation"})


model_item_mappings = run.register_model(model_name='item_to_veh_mappings', 
                           model_path=f'{output_folder}/{item_mapping_name}.dat',
                           tags={"data": "vehicle_clicks", "model": "recommendation", "model_name":model_name})

model_veh_mappings = run.register_model(model_name='veh_to_item_mappings', 
                           model_path=f'{output_folder}/{veh_mapping_name}.dat',
                           tags={"data": "vehicle_clicks", "model": "recommendation", "model_name":model_name})

In [ ]:
print(model_gnn.name, model_gnn.id, model_gnn.version, sep='\t')